<a href="https://colab.research.google.com/github/mshumer/gpt-pro-mode/blob/main/GPT_5_Pro_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Made by Matt Shumer ([@mattshumer_](https://x.com/mattshumer_) on X). Part of the [gpt-pro-mode](https://github.com/mshumer/gpt-pro-mode) repo.

In [ ]:
# @title Run To Set Up Pro Mode (OpenAI Responses API, GPT-5)
!pip -q install openai

from typing import List, Dict, Any
import time, os, concurrent.futures as cf
from openai import OpenAI
from dotenv import load_dotenv

MODEL = "gpt-5"
MAX_OUTPUT_TOKENS = 30000

def _extract_text(resp) -> str:
    if hasattr(resp, "output_text") and resp.output_text:
        return resp.output_text
    parts: List[str] = []
    for item in getattr(resp, "output", []) or []:
        for c in getattr(item, "content", []) or []:
            if getattr(c, "type", None) in ("output_text", "text"):
                parts.append(getattr(c, "text", ""))
    return "".join(parts).strip()

def _one_completion(client: OpenAI, prompt: str, *, effort: str = "high") -> str:
    delay = 0.5
    for attempt in range(3):
        try:
            # NOTE: no temperature/top_p here; reasoning models ignore/ban them
            resp = client.responses.create(
                model=MODEL,
                input=prompt,
                max_output_tokens=MAX_OUTPUT_TOKENS,
                # pass experimental/less-typed fields via extra_body
                extra_body={"reasoning": {"effort": effort}}
            )
            return _extract_text(resp)
        except Exception:
            if attempt == 2: raise
            time.sleep(delay); delay *= 2

def _build_synthesis_inputs(cands: List[str]) -> tuple[str, str]:
    numbered = "\n\n".join(f"<cand {i+1}>\n{t}\n</cand {i+1}>" for i, t in enumerate(cands))
    instructions = ("You are an expert editor. Synthesize ONE best answer from the "
                    "candidate answers provided, merging strengths, correcting errors, "
                    "and removing repetition. Do not mention the candidates or the process.")
    user = f"You are given {len(cands)} candidate answers delimited by <cand i> tags.\n\n{numbered}\n\nReturn the single best final answer."
    return instructions, user

def pro_mode(prompt: str, n_runs: int, openai_api_key: str | None = None) -> Dict[str, Any]:
    assert n_runs >= 1
    if openai_api_key: os.environ["OPENAI_API_KEY"] = openai_api_key
    client = OpenAI()

    # Fan out: keep some diversity by lightly varying *instructions*, not temperature
    STYLE_JITTER = [
        "Start with the concrete example, then generalize.",
        "Use a numbered outline and keep sentences ≤20 words.",
        "Compare two approaches and conclude with tradeoffs.",
        "Explain for a grad-level RL reader; be concise.",
        "Teach via a step-by-step walkthrough, then summarize."
    ]
    prompts = [f"{STYLE_JITTER[i % len(STYLE_JITTER)]}\n\n{prompt}" for i in range(n_runs)]

    cands: List[str] = [""] * n_runs
    with cf.ThreadPoolExecutor(max_workers=min(n_runs, 16)) as ex:
        fut2i = {ex.submit(_one_completion, client, p, effort="high"): i for i, p in enumerate(prompts)}
        for fut in cf.as_completed(fut2i):
            cands[fut2i[fut]] = fut.result()

    instr, user = _build_synthesis_inputs(cands)
    final_resp = client.responses.create(
        model=MODEL,
        instructions=instr,
        input=user,
        max_output_tokens=MAX_OUTPUT_TOKENS,
        extra_body={"reasoning": {"effort": "high"}}  # spend more thinking on the merge
    )
    return {"final": _extract_text(final_resp), "candidates": cands}




In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY environment variable is not set")

PROMPT = "Explain self-play in reinforcement learning with a concrete example."
NUMBER_OF_CANDIDATES = 5

try:
    result = pro_mode(PROMPT, NUMBER_OF_CANDIDATES, openai_api_key=os.getenv("OPENAI_API_KEY"))
    print("\n=== FINAL ===\n", result["final"])
    print("\n=== CANDIDATES ===")
    for i, c in enumerate(result["candidates"], 1):
        print(f"\n--- Candidate {i} ---\n{c}")
except Exception as e:
    print(f"\nError occurred: {type(e).__name__}")
    print(f"Error details: {str(e)}")


